In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('hometemp.ini')
import psycopg2
import time
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text, insert, inspect, exc, Table, Column, MetaData, Integer, DECIMAL, TIMESTAMP



In [ ]:
def _create_table(connection, table_name="sensor_data"):
    metadata = MetaData() 
    table_schema = Table(table_name, metadata, 
        Column('id', Integer, primary_key=True, autoincrement=True),
        Column('timestamp', TIMESTAMP(timezone=True), nullable=False),
        Column('humidity', DECIMAL, nullable=False),
        Column('room_temp', DECIMAL, nullable=False),
        Column('cpu_temp', DECIMAL, nullable=False))
    try:
        metadata.create_all(connection)
        print(f"Table '{table_name}' created successfully.")

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))

def _remove_table(connection, table_name=None):
    try:
        table = Table(table_name, MetaData(), autoload_with=connection)
        with connection.begin() as con:
            table.drop(con)
            print(f"Table '{table_name}' removed successfully.")

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))


def _get_table_size(connection, table_name="sensor_data"):
    try:
        with connection.connect() as con:
            result = con.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
            return int(result.scalar())

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))
        return -1


def _clear_table(connection, table_name="sensor_data"):
    try:
        table = Table(table_name, MetaData(), autoload_with=connection)
        with connection.begin() as con :
            con.execute(table.delete())
            print(f"Table '{table_name}' cleared successfully.")

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))

def _check_table_existence(connection, table_name="sensor_data"):
    try:
        return inspect(connection).has_table(table_name)

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))
        return False
    

def _init_db():
    auth = config["db"]
    db_url = f"postgresql://{auth['db_user']}:{auth['db_pw']}@{auth['db_host']}:{auth['db_port']}/{auth['db_name']}"
    try:
        return create_engine(db_url)
    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))


def init_db_connection(table_name="sensor_data"):
    """
    Establish the connection to postgres database and creates table it not existent.
    """
    try:
        con = _init_db()
        print("Connected to the database!")
        if not _check_table_existence(con, table_name):
            _create_table(con, table_name)
        return con
    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))



def insert_measurements_into_db(connection, timestamp, humidity, room_temp, cpu_temp, table_name="sensor_data"):
    try:
        table = Table(table_name, MetaData(), autoload_with=connection, extend_existing=True)
        with connection.begin() as con:
             data_to_insert = {
                 'timestamp': timestamp,
                 'humidity': humidity,
                 'room_temp': room_temp,
                 'cpu_temp': cpu_temp
             }
             insert_statement = insert(table).values(**data_to_insert)   
             con.execute(insert_statement)

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))


def read_data_into_dataframe(connection, table_name="sensor_data"):
    try:
        df = pd.read_sql(f"SELECT * FROM {table_name}", connection)
        return df

    except exc.SQLAlchemyError as e:
        print("Error: " + str(e))
        return None

In [ ]:
con = init_db_connection()
df = read_data_into_dataframe(con)
df['timestamp'] = df['timestamp'].map(lambda x : datetime.strptime(str(x).strip(), '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by='timestamp')
df = df.reset_index()
#print(str(df['timestamp'][0].month), str(df['timestamp'][0].day))

df

In [ ]:
# measurement data collection started on 12th august 2023, 11 hours 32 minutes 54 seconds
assert df['timestamp'][0].day == 12 and df['timestamp'][0].month == 8 and df['timestamp'][0].year == 2023 and df['timestamp'][0].hour == 11 and df['timestamp'][0].minute == 32 and df['timestamp'][0].second == 54

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [ ]:
def draw_plots(df, show_heatmap=True):
    subplots = 3 if show_heatmap else 2
    sns.set_theme(style="darkgrid")#sns.set(style="whitegrid")
    fig = plt.figure(figsize=(25, 12))
    gs = fig.add_gridspec(2, 2, height_ratios=[2, 2])  # 2 rows, 1 column

    # Temperature Measurements
    plt.subplot(gs[0])
    sns.lineplot(label="Home", x="timestamp", y="room_temp", data=df)
    plt.title("Temperature Over Time")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)
    #plt.tight_layout()
    #plt.show()

    # Humidity Measurement
    plt.subplot(gs[1])
    sns.lineplot(x="timestamp", y="humidity" , data=df, color='purple')
    plt.title("Humidity Over Time")
    plt.xlabel("Time")
    plt.ylabel("Humidity (%)")
    plt.xticks(rotation=45, ha='right')
    plt.gca().xaxis.grid(True)
    plt.gca().set_facecolor('#f5f5f5')
    sns.despine(left=True, bottom=True)

    df_last_24h = df[df["timestamp"] >= datetime.now() - timedelta(hours=25)]

    # Temperature Measurements last 24 h
    plt.subplot(gs[2])
    sns.lineplot(label="Home", x="timestamp", y="room_temp",marker='o',markersize=6, data=df_last_24h)
    plt.title("Temperature Last 24 Hours")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)

    # Humidity Measurements last 24 h
    plt.subplot(gs[3])
    sns.lineplot(x="timestamp", y="humidity", marker='o', markersize=6, color='purple', data=df_last_24h)
    plt.title("Humidity Last 24 Hours")
    plt.xlabel("Time")
    plt.ylabel("Humidity (%)")
    plt.xticks(rotation=45, ha='right')
    plt.gca().xaxis.grid(True)
    plt.gca().set_facecolor('#f5f5f5')
    sns.despine(left=True, bottom=True)

    # Correlations
    if show_heatmap:
        plt.subplot(1, subplots, 3) 
        #mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        #sns.heatmap(corr_matrix, annot=True,mask=mask, cmap="coolwarm", vmin=-1, vmax=1)
        sns.heatmap(df[["humidity", "room_temp", "cpu_temp"]].corr(), annot=True, cmap="coolwarm")
        plt.title("Correlation Heatmapfor All Categories")

    plt.tight_layout()
    name = datetime.now().strftime("%d-%m-%Y")
    #plt.savefig(f"plots/{name}.pdf")
    plt.show()
    plt.close()
    sns.reset_defaults()

draw_plots(df, False)

In [ ]:
plt.figure(figsize=(25, 8))
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
plt.title("Temperature Over Time")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(25, 8))
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
plt.title("Temperature Last 24h")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
csv_file = 'fitness.csv'  
fitness = pd.read_csv(csv_file, parse_dates=['time'])
fitness.describe()

In [ ]:
def plot_fitness(fitness):
    plt.figure(figsize=(25, 10))

    # Visualization 1: Line plot for weight over time
    plt.subplot(2, 2, 1)
    sns.lineplot(data=fitness, x='time', y='weight')
    plt.xticks(rotation=45)
    plt.title('Weight over Time')

    # Visualization 2: Line plot for BMI over time
    plt.subplot(2, 2, 2)
    sns.lineplot(data=fitness, x='time', y='bmi')
    plt.xticks(rotation=45)
    plt.title('BMI over Time')

    # Visualization 3: Line plot for body fat over time
    plt.subplot(2, 2, 3)
    sns.lineplot(data=fitness, x='time', y='body_fat')
    plt.xticks(rotation=45)
    plt.title('Body Fat over Time')

    # Visualization 4: Line plot for muscle over time
    plt.subplot(2, 2, 4)
    sns.lineplot(data=fitness, x='time', y='muscle')
    plt.xticks(rotation=45)
    plt.title('Muscle over Time')

    plt.tight_layout()
    plt.show()

def plot_fitness_corr(fitness):
    correlation_matrix = fitness[["weight", "bmi", "body_fat", "muscle", "kcal", "visceral"]].corr()
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=True,mask=mask, cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap')
    plt.show()

#plot_fitness(fitness)
#plot_fitness_corr(fitness)